now I want to go to the website so let;s Nike has this particular job posting I want to scrap the data from this website first if you look at the Lang chain documentation it has this class called WebBaseLoader
what this class will do is it . it will accept the URL of that job posting and  it will extract data from that so it's basically web scraping 

In [3]:

from langchain_groq import ChatGroq

In [4]:
from langchain_groq import ChatGroq
llm= ChatGroq(
    temperature=0,   
    groq_api_key="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",
    model_name="llama-3.1-70b-versatile"
)
response = llm.invoke("The First person to land on moon was:..")
print(response.content)

Neil Armstrong. He became the first person to set foot on the Moon on July 20, 1969, during the Apollo 11 mission.


In [5]:
pip install langchain_community


Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install langchain


Note: you may need to restart the kernel to use updated packages.


using WebBaseLoader it went to this website the data will be in HTML format internally it went through that Html structure and extracted all this key information

In [13]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-38030?from=job%20search%20funnel")
page_data = loader.load().pop().page_content
print(page_data)

Apply for NIKE, Inc. Data & Machine Learning Engineer Undergraduate Internship

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsInternshipsNIKE, Inc. Data & Machine Learning Engineer Und

now we will  write a prompt for our llm for which we will import this prompt template class

In [14]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
print(res)
type(res.content)

content='```json\n{\n  "role": "NIKE, Inc. Data & Machine Learning Engineer Undergraduate Internship",\n  "experience": "0-2 years",\n  "skills": [\n    "Python",\n    "SQL",\n    "Tableau",\n    "AI/ML techniques (neural networks, tree ensembles, regressions, etc.)",\n    "Data structures and algorithms",\n    "Solution and technical design",\n    "Problem-solving, debugging, testing, and analytical skills"\n  ],\n  "description": "Apply machine learning and engineering expertise to improve existing algorithms and understand and be able to leverage core concepts of AI/ML to research and develop new algorithms. Collaborate in multi-functional teams to support other groups who will also be working on science or science-engineering-related work."\n}\n```' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 164, 'prompt_tokens': 1528, 'total_tokens': 1692, 'completion_time': 0.657081955, 'prompt_time': 0.408754785, 'queue_time': 0.005499138999999986, 'total_time':

str

I want to convert this text format into JSON format

In [15]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'NIKE, Inc. Data & Machine Learning Engineer Undergraduate Internship',
 'experience': '0-2 years',
 'skills': ['Python',
  'SQL',
  'Tableau',
  'AI/ML techniques (neural networks, tree ensembles, regressions, etc.)',
  'Data structures and algorithms',
  'Solution and technical design',
  'Problem-solving, debugging, testing, and analytical skills'],
 'description': 'Apply machine learning and engineering expertise to improve existing algorithms and understand and be able to leverage core concepts of AI/ML to research and develop new algorithms. Collaborate in multi-functional teams to support other groups who will also be working on science or science-engineering-related work.'}

In [16]:
type(json_res)

dict

now next step is to prepare chroma DB so I have Csv file  where I have all the technologies that my company
is working on so atliq is working on all these compaines 

here i am using some dummy URl  for each of the technologies these are the portfolio links  and I need to insert this thing into chroma Db then whenever there is a job posting
should extract the skills from the job and it will match one or multiple of these technologies and then it should retrieve this portfolio URLs which I will use in my email all  right so I have to put CSV file here and I'm going to just import that here so import Panda as pd and i will create a data frame 

In [24]:
import pandas as pd
file_path=r"C:\Users\naini\OneDrive\Desktop\llm\my_portfolio.csv"
df = pd.read_csv(file_path)
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


and you will insert your data one by one into chroma DB so since we have writtten thihs code before I'm going to save you some time imported chroma DB instead of client see in our sample notebook we use client correct when you use client it wwill create the chroma DB in memory but I will use persistent client so it will create DB on a disk so in your current folder no folder for chroma Db when I execute this code it will ccreate that database store it in a folder so that next time when I want to retrieve my data I have it on the disk
if not collection.count means if let's say collection doesn't have any cout if it is being created for the first time then you iterate through all your data frame rows folks this is very very easy simple python code you just iterating over data frame rows and for each of the rows you are adding collection.add you are adding a document metadata is your link and text tag is your main document



In [25]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

so if you look at now your folder you see this vector store folder is created so internally it is storing the data here you don't need to look at these files (behind the folder name vector store )
but it shows that when you use a persistent client it will create a folder and store your records inside that folder now I will do simple collection Query I will say Okay I need a person with experience in python experience in react and when I do that see for the first one experience in python it is bringing thses two  which are python related portfolio

In [27]:
job = {'skills': ['Python', 'Data Analysis']}


In [28]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-portfolio'}]]

In [29]:
links = collection.query(query_texts=['Experience in Python', 'Expertise in react'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}]]

In [30]:
links = collection.query(query_texts=['Experience in Python', 'Expertise in react Native'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-native-portfolio'},
  {'links': 'https://example.com/react-portfolio'}]]

In [35]:
job

{'role': 'NIKE, Inc. Data & Machine Learning Engineer Undergraduate Internship',
 'experience': '0-2 years',
 'skills': ['Python',
  'SQL',
  'Tableau',
  'AI/ML techniques (neural networks, tree ensembles, regressions, etc.)',
  'Data structures and algorithms',
  'Solution and technical design',
  'Problem-solving, debugging, testing, and analytical skills'],
 'description': 'Apply machine learning and engineering expertise to improve existing algorithms and understand and be able to leverage core concepts of AI/ML to research and develop new algorithms. Collaborate in multi-functional teams to support other groups who will also be working on science or science-engineering-related work.'}

In [31]:
job=json_res
job['skills']

['Python',
 'SQL',
 'Tableau',
 'AI/ML techniques (neural networks, tree ensembles, regressions, etc.)',
 'Data structures and algorithms',
 'Solution and technical design',
 'Problem-solving, debugging, testing, and analytical skills']

In [32]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/flutter-portfolio'},
  {'links': 'https://example.com/android-tv-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/typescript-frontend-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}]]

In [34]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Patel Nainisha, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Data & Machine Learning Engineering Solutions for NIKE, Inc.

Dear Hiring Manager,

I came across the NIKE, Inc. Data & Machine Learning Engineer Undergraduate Internship job description and was impressed by the company's commitment to leveraging AI/ML techniques to drive innovation. As a Business Development Executive at AtliQ, I'd like to introduce our company as a potential partner in fulfilling your data and machine learning engineering needs.

At AtliQ, we specialize in providing tailored AI and software consulting solutions that empower businesses to scale, optimize processes, reduce costs, and enhance overall efficiency. Our team of experts has extensive experience in developing and implementing machine learning algorithms, data structures, and technical design solutions.

Our portfolio showcases our capabilities in machine learning and Python development, which aligns perfectly with the skills required for the internship. I'd like to highlight the following rele